In [93]:
import pandas as pd
import numpy as np

In [94]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
labels = pd.read_csv("test_labels.csv")

In [95]:
#train.head()
#labels

In [96]:
#train.isnull().any(),test.isnull().any()

In [97]:
classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[classes].values
list_sentences_train = train["comment_text"]
list_sentences_test = test["comment_text"]

In [98]:
from keras.preprocessing.text import Tokenizer

In [99]:
max_features = 20000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)

In [100]:
from keras.preprocessing.sequence import pad_sequences    #用于填充序列为maxlen：keras只能接受长度相同的序列输入。因此如果目前序列长度参差不齐，这时需要使用pad_sequences()。该函数是将序列转化为经过填充以后的一个长度相同的新序列新序列。

In [101]:
maxlen = 200
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)    #train set
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)    #test set

In [102]:
#X_te.shape
#X_te

In [103]:
#transfer to dataframe
X_t1 = pd.DataFrame(X_t)
#X_t1
X_te1 = pd.DataFrame(X_te)
#X_te1
y_data = pd.DataFrame(y)
#y_data

In [104]:
y_toxic = y_data[0]
y_severe_toxic = y_data[1]
y_obscene = y_data[2]
y_threat = y_data[3]
y_insult = y_data[4]
y_identity_hate = y_data[5]

In [105]:
def sigmoid(z):
    temp = 1+np.exp(-z)
    result = 1/temp
    return result

In [106]:
def cost(theta, X, y):
    ''' cost function for you to minimize'''
    # use dot() function to calculate matrices multiplication
    theta_x = np.dot(theta, X.transpose())
    h_x = []
    for i in theta_x:
        h_x.append(sigmoid(i))
    cost = []
    for i in range(len(y)):
        cost.append(-y[i]*np.log(h_x[i])-(1-y[i])*np.log(1-h_x[i]))
    result = sum(cost) / len(y)
    return result

In [107]:
#add 1
X_t1.insert(0, 'Ones', 1)
#X_t1

In [108]:
theta = np.array(np.zeros(201))
X_t1.shape, theta.shape, y_toxic.shape

((159571, 201), (201,), (159571,))

In [129]:
#cost(theta, X_t1, y_toxic)
cost(theta, X_t1, y_severe_toxic)

0.6931471805585643

In [130]:
def gradient(theta, X, y):
    theta_x = np.dot(theta, X.transpose())
    h_x = sigmoid(theta_x)
    grad = (1/len(y))*np.dot(h_x-y.transpose(), X)
    return grad

In [136]:
#gra1 = gradient(theta, X_t1, y_toxic)
#gra1
#gra2 = gradient(theta, X_t1, y_severe_toxic)
#gra2

In [137]:
import scipy.optimize as opt
#x0 = np.asarray(gra1)
#x0_severe_toxic = np.asarray(gra2)
#x0_severe_toxic

In [121]:
#千万不要重新跑
result = opt.minimize(fun=cost, x0=theta, args = (X_t1, y_toxic), method='TNC', jac = gradient)
result

     fun: 0.3107386230816921
     jac: array([ 1.71777422e-05,  5.25880591e-04,  1.09716811e-04, -1.34083843e-04,
        1.70339891e-03,  9.90386419e-04,  1.10455517e-03,  1.04027666e-03,
        1.13553807e-03, -2.60143221e-05,  1.28213488e-03,  1.49997762e-03,
        1.14241295e-03,  5.71953911e-04,  1.62638116e-03,  8.12609678e-04,
        7.40423168e-04,  1.10333373e-03,  5.71066601e-04,  5.80147035e-04,
        2.66417177e-04,  5.29709800e-04,  1.31606521e-04, -4.86134449e-04,
        3.75166152e-04,  3.73589356e-04,  4.10358819e-04,  1.39403988e-03,
        4.54060736e-04,  4.08167356e-04,  2.75419874e-04,  2.31126642e-05,
        7.49324649e-04,  1.85079244e-04, -3.80842942e-04,  4.93572979e-04,
        4.82231131e-04,  4.86114770e-04, -5.39032181e-04,  7.84681088e-04,
       -5.54156672e-04,  7.20094404e-04,  6.95126945e-04,  5.86219700e-04,
        1.91443877e-04,  8.56777765e-05,  3.48547625e-04,  2.46417863e-04,
        4.27395194e-04,  1.15692047e-03,  3.88155455e-04,  8.

In [ ]:
#千万不要重新跑 y_severe_toxic
result_2 = opt.minimize(fun=cost, x0=theta, args = (X_t1, y_severe_toxic), method='TNC', jac = gradient)
result_2

<ipython-input-106-961381ccff74>:10: RuntimeWarning: divide by zero encountered in log
  cost.append(-y[i]*np.log(h_x[i])-(1-y[i])*np.log(1-h_x[i]))
<ipython-input-106-961381ccff74>:10: RuntimeWarning: invalid value encountered in multiply
  cost.append(-y[i]*np.log(h_x[i])-(1-y[i])*np.log(1-h_x[i]))


In [124]:
cost(result.x, X_t1, y_toxic)

0.3107386230816921

In [125]:
def predict(theta, X):
    #theta_x = np.dot(theta, X.transpose())
    #print(theta_x)
    probability = sigmoid(np.dot(theta, X.transpose()))
    #print(probability)
    return (probability >= 0.5).astype(int)

In [128]:
from sklearn.metrics import classification_report  # this is a convinient function to show the classification performance

theta_min =  result.x
y_toxic_pred = predict(theta_min, X_t1)
print(classification_report(y_toxic, y_toxic_pred))

              precision    recall  f1-score   support

           0       0.90      1.00      0.95    144277
           1       0.50      0.00      0.00     15294

    accuracy                           0.90    159571
   macro avg       0.70      0.50      0.47    159571
weighted avg       0.87      0.90      0.86    159571

